# IBM Advanced Data Science Capstone
## Extract, Transform, Load
ETL is the discipline of transfering data from one or more sources combined into a usable set of data into a destination system. It also involves some preliminary cleaning and data transformation. Mostly, I will focus on collecting nessecary data, combining into one, and preparing for the next step.

In [26]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pandasql import sqldf
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
import plotly.graph_objects as go
import plotly.express as px
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from time import sleep
import re

In [2]:
data = pd.read_parquet('data/vehicles.parquet.gzip')

In [3]:
def get_capital_lat_long():
    # Get state capital data from people.sc.fsu.edu
    capitals_ll = requests.get('https://people.sc.fsu.edu/~jburkardt/datasets/states/state_capitals_ll.txt')
    # Data is split by new row, value lowered, and converted to dataframe
    capitals_ll = capitals_ll.text.split('\n')
    capitals_ll = [c.lower().split() for c in capitals_ll]
    capitals_ll = pd.DataFrame(
        data=capitals_ll,
        columns=['capital', 'lat', 'long']
    )
    # Set lat and long data type to float
    capitals_ll[['lat', 'long']] = capitals_ll[['lat', 'long']].apply(lambda x: x.astype(float))
    # Convert lat long to shapely.geometry.Point
    geometry = [Point(xy) for xy in zip(capitals_ll['long'], capitals_ll['lat'])]
    # Output geopandas dataframe
    capitals_ll = gpd.GeoDataFrame(
        data=capitals_ll.drop(['lat', 'long'], axis=1),
        crs={'init': 'epsg:4326'},
        geometry=geometry
    )
    
    return capitals_ll.dropna(subset=['capital'])

In [4]:
def get_state_neighbors():
    capitals_ll = get_capital_lat_long()

    # Get list of neighboring states for each state
    state_neighbors = requests.get('https://people.sc.fsu.edu/~jburkardt/datasets/states/state_neighbors.txt')
    # Perform same transformations as before
    state_neighbors = state_neighbors.text.split('\n')
    state_neighbors = [sn.lower().split() for sn in state_neighbors]
    max_neighbors = max([len(sn) for sn in state_neighbors])
    
    states = ['capital']
    for i in range(max_neighbors-1):
        states.append(f'capital_neighbor_{i}')
    
    state_neighbors = pd.DataFrame(
        data=state_neighbors,
        columns=states
    )

    state_neighbors = state_neighbors \
        .dropna(subset=['capital'])

    # Change neighbor states to state capitals lat/long Points
    for i, state in enumerate(states):
        state_neighbors = state_neighbors \
            .merge(capitals_ll, how='left', left_on=state, right_on='capital') \
            .rename(columns={'geometry': f'cap_{i}'})

    keep_cols = [col for col in state_neighbors.columns if col[:7] != 'capital']
    
    # We still need state abbreviations for main dataset
    output = capitals_ll \
        .drop('geometry', axis=1) \
        .merge(state_neighbors[keep_cols], how='left', left_index=True, right_index=True)

    return output

In [5]:
def haversine_distance(point1, point2):
    # Check first if one of the points are blank. If not, continue, else return nan.
    if point1 and point2:
        # Following formula from https://en.wikipedia.org/wiki/Haversine_formula
        lon1, lat1, lon2, lat2 = map(np.radians, [point1.x, point1.y, point2.x, point2.y])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
        c = 2*np.arcsin(np.sqrt(a))
        km = 6371*c
        return km
    else:
        return np.nan

In [6]:
# Convert main data lat and long to Points and append to data.
# Hereafter, the output from get_state_neighbors() is joined onto the main data.
geometry = [Point(xy) for xy in zip(data['long'], data['lat'])]
g_data = gpd.GeoDataFrame(
    data=data,
    crs={'init': 'epsg:4326'},
    geometry=geometry
).merge(
    get_state_neighbors(),
    how='left',
    left_on='state',
    right_on='capital'
)

In [7]:
# Calculating haversine distances between vehicle state and (neighboring) capital(s).
capitals = [col for col in g_data.columns.values if (col[:3] == 'cap') & (col != 'capital')]
for i, capital in enumerate(capitals):
    g_data[f'distance_{i}'] = g_data.apply(lambda x: haversine_distance(x['geometry'], x[capital]), axis=1)
    g_data = g_data.drop(capital, axis=1)

In [8]:
g_data.describe()

,price,year,odometer,lat,long,distance_0,distance_1,distance_2,distance_3,distance_4,distance_5,distance_6,distance_7,distance_8
count,265866.000000,265855.000000,2.261230e+05,264114.000000,264114.00000,264114.000000,259629.000000,257845.000000,225516.000000,171338.000000,101656.000000,59050.000000,14122.000000,10694.000000
mean,15669.512687,2010.307596,9.916456e+04,38.826539,-93.80658,249.848497,495.211429,505.351872,549.721586,542.868669,495.771869,553.028821,555.333489,564.272050
std,11917.514833,9.171975,1.081257e+05,5.945867,17.77061,364.156530,353.449924,364.040311,423.605099,381.153040,376.088126,341.395884,287.790485,206.546242
min,2000.000000,1911.000000,0.000000e+00,-80.386400,-161.39400,0.015682,0.154056,1.181180,1.497300,0.095287,0.546068,0.830410,35.914750,29.146037
25%,6975.000000,2007.000000,4.621050e+04,35.119200,-106.49800,73.378253,291.682487,268.202785,285.286762,285.672330,262.684336,366.928001,338.462465,464.446351
50%,12500.000000,2012.000000,9.300000e+04,39.462150,-87.90650,168.890080,427.663599,464.152595,460.345733,440.507785,403.915371,501.461818,558.526287,544.622575
75%,20495.000000,2016.000000,1.374955e+05,42.598200,-81.00790,301.114559,629.837336,637.688634,714.240088,722.553522,638.306018,681.103694,702.232763,689.066750
max,78900.000000,2021.000000,9.999999e+06,80.383400,115.52400,15903.621465,15893.018748,15563.655551,15610.385172,15352.239521,11951.400180,12518.862660,12197.275975,6701.692403


We'll create an extra feature from this, which is going to be the average distance from the data latitude and longitude to the top three nearest state capitals.

In [9]:
def get_average_state_distance():
    state_neighbors = get_capital_lat_long()
    n = len(state_neighbors[['capital', 'geometry']])
    output = []
    for i in range(n):
        state = state_neighbors['capital'][i]
        capital = state_neighbors['geometry'][i]
        distances = []
        for j in range(n):
            if i != j:
                neighbor = state_neighbors['geometry'][j]
                distances.append(haversine_distance(capital, neighbor))
        output.append([state, np.mean(sorted(distances)[:3])])
    return pd.DataFrame(output, columns=['state', 'distance'])

In [10]:
g_data = g_data.merge(
    get_average_state_distance(),
    how='left',
    left_on='state',
    right_on='state'
).drop('geometry', axis=1)

Before we move on to further external features, let's also do something about a few other, so far, categorical features. Namely `cylinders`, `condition`, `size`, and `title_status`. The `cylinders` column can already be considered as continuous, where higher value is "better". We'll simply remove the text 'cylinders' and convert to integer:

In [11]:
g_data['cylinders'] = pd.to_numeric(g_data['cylinders'].str.replace(r'[^0-9]', '').str.strip())

### Additional External Features
Let's find some more information about our cars. Luckily, the British market research site YouGov provides some exciting information, ranking car brands by popularity and fame, grouped by eg. sex and age group. We'll be focusing solely on an overall rating. Using [Selenium](https://selenium-python.readthedocs.io/) in combination with [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) we can scrape the site for information. We need to make the website think it's accessed by an actual browser. We'll use Google Chrome for this:

In [12]:
# Perusing yougov.com, we expect at least fifty entries. 
# This will keep running, until this condition is satisfied
rows = 0
while rows < 50:
    CHROME_PATH = 'C:\Program Files%s\Google\Chrome\Application\chrome.exe'
    CHROMEDRIVER_PATH = 'chromedriver\chromedriver.exe'

    # Set some options (don't open the actual browser)
    chrome_options = Options()
    chrome_options.add_argument('--headless')

    try:
        chrome_options.binary_location = CHROME_PATH % ''
        driver = webdriver.Chrome(
            executable_path=CHROMEDRIVER_PATH,
            chrome_options=chrome_options
        )
    except:
        chrome_options.binary_location = CHROME_PATH % ' (x86)'
        driver = webdriver.Chrome(
            executable_path=CHROMEDRIVER_PATH,
            chrome_options=chrome_options
        )
    # Get popularity
    driver.get('https://today.yougov.com/ratings/automotive/popularity/car-makers/all')
    for _ in range(20):
        # All rankings only becomes visible if we scroll down
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(0.1)
    soup = BeautifulSoup(driver.page_source)

    ratings = []
    # Search for html tags in the soup and save to array
    for s in soup.find_all('li', attrs={'class': 'ng-star-inserted'}):
        if s.find('h2') is not None:
            brand = s.find('h2').contents[0].strip().lower()
            for x in s.find_all('span', attrs={'_yg-server-content-sc67': ''}):
                if len(x.contents) > 1:
                    rating = x.contents[0].strip()
            ratings.append([brand, float(rating)])

    # Get fame
    driver.get('https://today.yougov.com/ratings/automotive/fame/car-makers/all')
    for _ in range(20):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(0.1)
    soup = BeautifulSoup(driver.page_source)

    fames = []
    # Search for html tags in the soup and save to array
    for s in soup.find_all('li', attrs={'class': 'ng-star-inserted'}):
        if s.find('h2', attrs={'class': 'title'}) is not None:
            brand = s.find('h2', attrs={'class': 'title'}).contents[0].strip().lower()
            for x in s.find_all('span', attrs={'_ngcontent-yg-app-c53': ''}):
                if len(x.contents) > 1:
                    fame = float(x.contents[0].strip())
            fames.append([brand, fame])

    driver.close()

    # Combine the two results into a dataframe
    popularity = pd.DataFrame(ratings, columns=['manufacturer', 'popularity']) \
        .set_index('manufacturer') \
        .merge(
            pd.DataFrame(fames, columns=['manufacturer', 'fame']) \
                .set_index('manufacturer'),
            how='outer',
            left_index=True,
            right_index=True
        ) \
        .reset_index()

    rows = popularity.dropna().shape[0]

In [13]:
popularity.shape

(57, 3)

In [14]:
# Join the popularity factors onto the main data
g_data = g_data.merge(
    popularity,
    how='left',
    left_on='manufacturer',
    right_on='manufacturer'
)

The last columns we can infer additional information from are the `vin` and `description` columns. VIN (Vehicle Idintification Number) is a unique identifier every vehicle has, and from this information about make, model, engine size, equipment, etc. can be obtained. There is a wealth of online VIN decoders out there, some free but most requires a paid subscription. Fortunately the [NHTSA](https://www.nhtsa.gov/) (National Highway Traffic Saftety Administration) offers up a free API to decode VIN's in batches. Unfortunately it takes a lot of time (roughly 4 hours for this amount of data), so I have provided the code to obtain decoded VIN's below:

In [15]:
def vin_decoder(data):
    # Link to api for batch lookups
    URI = 'https://vpic.nhtsa.dot.gov'
    endpoint = f'/api/vehicles/DecodeVINValuesBatch/?format=json'
    # Inject VINs a json 
    data = {'data': ';'.join(data)}
    # Get response and convert to dataframe
    r = requests.post(URI + endpoint, data=data)
    j = json.loads(r.content)
    # Remove redundant columns
    df = pd.DataFrame(j['Results']) \
        .replace(r'(^\s*$)|(Not Applicable)', np.nan, regex=True) \
        .dropna(how='all', axis=1)
    return df

In [16]:
# Get unique VINs
vins = g_data['vin'].dropna().unique()

# Wrap VINs into batches of 50 and lookup on NHTSA
vin_df = pd.DataFrame()
for i in range(0, len(vins_missing), 50):
    vin_df = vin_df.append(vin_decoder(data=vins_missing[i:i+50])

# Checkpoint: Save the data
vin_df.to_parquet('data/vin_decoded.parquet.gzip', compression='gzip'))

Decoded VINs checkpoint!

In [17]:
vin_df = pd.read_parquet('data/vin_decoded.parquet.gzip')

The information obtained from NHTSA yields some interesting additional information. Let's first sort through the observations and only consider the columns with less than 50% missing values:

In [18]:
vin_df_nans = vin_df.isna().sum().sort_values(ascending=False)/vin_df.shape[0]
vin_df_nans[vin_df_nans < 0.5]

EngineKW               0.476979
EngineHP               0.476911
EngineModel            0.474641
EngineConfiguration    0.380934
GVWR                   0.368322
TPMS                   0.312859
Series                 0.309635
DriveType              0.309544
PlantState             0.286194
PlantCompanyName       0.280484
AirBagLocSide          0.228233
Doors                  0.197789
PlantCity              0.173667
SeatBeltsAll           0.135310
AirBagLocFront         0.133721
EngineCylinders        0.121949
PlantCountry           0.075090
FuelTypePrimary        0.071163
DisplacementCC         0.037608
DisplacementCI         0.037608
DisplacementL          0.037608
BodyClass              0.028833
ModelID                0.028243
Model                  0.028243
ModelYear              0.025382
Manufacturer           0.024894
VehicleType            0.024894
ManufacturerId         0.024894
MakeID                 0.024894
Make                   0.024894
VIN                    0.016153
ErrorTex

From the above, going by rule of thumb, the features that are interesting are the following:

| Column | | Description | | Comments |
| :- | --- | :- | --- | :- |
| `EngineKW`/`EngineHP` | | Engine power in kW and horsepower | | This column has a high percentage of missings, but hopefully some information can be gained regardless, eg. by averaging over make and model |
| `EngineConfiguration` || Categorisation of the engines |||
| `GVWR` || Gross vehicle weight rating |||
| `TPMS` || Tire-pressure monitoring system |||
| `Doors` || Number of doors || Can be used as additional information on the size of the vehicle |
| `EngineCylinders` || Number of cylinders in engine || Should be the same as `cylinders` in the main data. We'll take it in case the main data is missing. |
| `DisplacementXX` || Engine size |||
| `ModelYear` || Model year || Should be the same as `year` in the main data, but we'll grab it for good measure |
| `VIN` || Vehicle identification number || Neccesary to combine this data with the main data |

Before merging dataframes, let's convert the columns into correct data types. `EngineKW`/`EngineHP`, `Doors`, `EngineCylinders`, `ModelYear`, and the `DisplacementXX` columns should be numerical, while the rest are categorical.

In [19]:
def preprocess_vin_df(df):
    num_cols = [
        'enginekw',
        'enginehp',
        'doors',
        'enginecylinders',
        'modelyear',
        'displacementcc',
        'displacementci',
        'displacementl'
    ]
    cat_cols = [
        'gvwr',
        'tpms',
        'engineconfiguration',
        'manufacturer',
        'make',
        'model',
        'vin'
    ]

    df.columns = [x.lower() for x in df.columns]
    df[num_cols] = df[num_cols].astype(float)
    df[cat_cols[:-1]] = df[cat_cols[:-1]].apply(lambda x: x.str.lower())
    df['gvwr'] = df['gvwr'].str.replace(r'(\:.*)', '')
    df = df[cat_cols + num_cols]
    df['vin2'] = df['vin'].str[3:10]
    
    return df

In [20]:
vin_df = preprocess_vin_df(vin_df)

As one last, final thing, we notice that the `model` column is very messy, eg. Ford F-150 can be listed multiple times with various extra text, which we'll assume is information on the model variant. The model of a vehicle is very important when determining the sales price. It has a big say wether it is a Ford Fiesta or a Ford Mustang, so keeping this feature is crucial. In order to simplify to the model level, we need to somehow fix this. One way would be to manually map every single unique model entry by hand. However, as we see below, there are over 20,000 unique entries in `model` (which if left untreated would essentially leave us with ~20,000 dummy variables, which adds too much complexity to the model), and it would therefore be tedious work. Another way is by being a bit clever. Not accounting for spelling errors, we should be safe to assume that a "mazda3 sport sedan 4d", which is in the Craigslist dataset, can be mapped simply to "mazda3", which we obtained from the VIN previously. The solution I propose is to loop over every model in every brand, and check if the VIN model is in the Craigslist model by comparing strings. If we can map the two, we keep it. Else, we set it to NaN.

In [30]:
g_data['model'].value_counts()

f-150               5081
silverado 1500      3338
escape              2738
1500                2453
camry               2357
                    ... 
tahoe lt custom        1
accord lx wagon        1
f-150 4wd lariat       1
supercab 4x4           1
s-10 blazer suv        1
Name: model, Length: 19027, dtype: int64

In [31]:
# Create empty dictionary for mapping
model_map = {}

# Find all unique brands in vin_df and loop
brands = vin_df['make'].unique()
brands = [str(b).lower() for b in brands if b is not None]
for brand in brands:
    # Set the value for the current brand as an empty dictionary
    model_map[brand] = {}

    # Find all unique models for currrent brand in vin_df
    models_vin = vin_df.loc[vin_df['make'].str.lower() == brand, 'model'].unique()
    models_vin = [str(m).lower() for m in models_vin if m is not None]
    # Find all unique models for currrent brand in Craigslist dataset
    models = g_data.loc[g_data['manufacturer'] == brand, 'model'].unique()

    # Loop over models in model_vin and compare
    for model in models_vin:
        _model = re.sub(r'[^0-9a-z]', '', model)
        for m in models:
            if re.sub(r'[^0-9a-z]', '', model) in re.sub(r'[^0-9a-z]', '', m):
                model_map[brand][m] = model

    # Map the dictionary on the model column for current brand
    g_data.loc[g_data['manufacturer'] == brand, 'model'] = g_data \
        .loc[g_data['manufacturer'] == brand, 'model'] \
        .map(model_map[brand])

In [32]:
g_data['model'].value_counts()

f-150                    10084
silverado                 9752
f-250                     4967
1500                      4792
sierra                    4436
                         ...  
v-rod                        1
lr2 awd                      1
davidson 72                  1
panemera 4                   1
range sport autobiogr        1
Name: model, Length: 885, dtype: int64

We reduced the number of unique models from 19,000 to just about 900. That's effective data cleaning!

According to [Wikipedia](https://en.wikipedia.org/wiki/Vehicle_identification_number#Components), digits 4 to 8(9) of the Vehicle Identification Number contains vehicle descripter features such as make, model, engine configuration, displacement, and so on. We'll create a new column with only those digits, which we'll use to help combine the newly obtained VIN information with the original data. Since not all vehicles in our original dataset have an associated VIN, we need to enrich these rows in some other way. Below is an SQL query, that calculates the averages of the numerical features of `vin_df` and ranks them according to the grouping parameters:

| Rank || Condition |
| :- | --- | :- |
| 1 || Direct match on `vin` |
| 2 || If no previous match, match on `vin2` |
| 3 || If no previous match, match on `make`, `model` and `year` |
| 4 || If no previous match, match on `make` and `model` |
| 5 || If no previous match, match on `make` |

Rank 1 and 2 are the best options. Rank 3 is good, but since the displacement and engine size most likely varies with variant and vehicle type, this introduces higher uncertainty that the match is correct. This will be the case as we lessen the constraints on the join parameters through ranks 4 and 5.

In [33]:
g_data['vin2'] = g_data['vin'].str[3:10]

In [34]:
pysqldf = lambda q: sqldf(q, globals())
g_data = pysqldf("""
    select
        g_data.*
      , t1.gvwr
      , t1.tpms
      , t1.engineconfiguration
      , case
            when length(g_data.model) <= length(coalesce(t1.model, t3.model, t4.model, g_data.model))
            then g_data.model
            else coalesce(t1.model, t3.model, t4.model, g_data.model)
        end as model_new
      , coalesce(t1.displacementcc, t2.displacementcc, t3.displacementcc, t4.displacementcc, t5.displacementcc) as displacementcc
      , coalesce(t1.displacementci, t2.displacementci, t3.displacementci, t4.displacementci, t5.displacementci) as displacementci
      , coalesce(t1.displacementl, t2.displacementl, t3.displacementl, t4.displacementl, t5.displacementl) as displacementl
      , coalesce(t1.enginekw, t2.enginekw, t3.enginekw, t4.enginekw, t5.enginekw) as enginekw
      , coalesce(t1.enginehp, t2.enginehp, t3.enginehp, t4.enginehp, t5.enginehp) as enginehp
      , coalesce(t1.enginecylinders, t2.enginecylinders, t3.enginecylinders, t4.enginecylinders, t5.enginecylinders) as enginecylinders
      , round(coalesce(t1.doors, t2.doors, t3.doors, t4.doors, t5.doors), 0) as doors
      , coalesce(t1.vin_avg_weight, t2.vin_avg_weight, t3.vin_avg_weight, t4.vin_avg_weight, t5.vin_avg_weight) as vin_avg_weight
    from g_data
	left join (
		select
			*
		  , 1 as vin_avg_weight
		from vin_df
	) t1
        on g_data.vin = t1.vin
	left join (
        select
            vin2
          , avg(displacementcc) as displacementcc
          , avg(displacementci) as displacementci
          , avg(displacementl) as displacementl
          , avg(enginekw) as enginekw
          , avg(enginehp) as enginehp
          , avg(enginecylinders) as enginecylinders
          , avg(doors) as doors
          , 2 as vin_avg_weight
        from vin_df
        where make is not null
        group by 
            make
    ) t2
        on g_data.vin2 = t2.vin2
    left join (
        select
            make
          , model
          , modelyear
          , avg(displacementcc) as displacementcc
          , avg(displacementci) as displacementci
          , avg(displacementl) as displacementl
          , avg(enginekw) as enginekw
          , avg(enginehp) as enginehp
          , avg(enginecylinders) as enginecylinders
          , avg(doors) as doors
          , 3 as vin_avg_weight
        from vin_df
        where make is not null
        group by 
            make
          , model
          , modelyear
    ) t3
        on g_data.manufacturer = t3.make
        and g_data.model = t3.model
        and g_data.year = t3.modelyear
    left join (
        select
            make
          , model
          , avg(displacementcc) as displacementcc
          , avg(displacementci) as displacementci
          , avg(displacementl) as displacementl
          , avg(enginekw) as enginekw
          , avg(enginehp) as enginehp
          , avg(enginecylinders) as enginecylinders
          , avg(doors) as doors
          , 2 as vin_avg_weight
        from vin_df
        where make is not null
        group by 
            make
          , model
    ) t4
        on g_data.manufacturer = t4.make
        and g_data.model = t4.model
    left join (
        select
            make
          , avg(displacementcc) as displacementcc
          , avg(displacementci) as displacementci
          , avg(displacementl) as displacementl
          , avg(enginekw) as enginekw
          , avg(enginehp) as enginehp
          , avg(enginecylinders) as enginecylinders
          , avg(doors) as doors
          , 1 as vin_avg_weight
        from vin_df
        where make is not null
        group by 
            make
    ) t5
        on g_data.manufacturer = t5.make
    
;
""")
g_data = g_data \
  .drop(['model', 'region', 'vin', 'vin2'], axis=1) \
  .rename(columns={'model_new': 'model'})

In [36]:
g_data.to_parquet('data/vehicles_etl.parquet.gzip', compression='gzip')